## LaRE 

In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [68]:
# imports
import os
import sys
sys.path.append(os.path.abspath(os.path.join("..")))
from models import get_model
from torchsummary import summary
import torch
from src.landmark_extraction import extract_rois

In [3]:
model_name = "CLIP:RN50"
model = get_model(model_name)

In [61]:
summary(model.clip_model.visual, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 112, 112]             864
       BatchNorm2d-2         [-1, 32, 112, 112]              64
              ReLU-3         [-1, 32, 112, 112]               0
            Conv2d-4         [-1, 32, 112, 112]           9,216
       BatchNorm2d-5         [-1, 32, 112, 112]              64
              ReLU-6         [-1, 32, 112, 112]               0
            Conv2d-7         [-1, 64, 112, 112]          18,432
       BatchNorm2d-8         [-1, 64, 112, 112]             128
              ReLU-9         [-1, 64, 112, 112]               0
        AvgPool2d-10           [-1, 64, 56, 56]               0
           Conv2d-11           [-1, 64, 56, 56]           4,096
      BatchNorm2d-12           [-1, 64, 56, 56]             128
             ReLU-13           [-1, 64, 56, 56]               0
           Conv2d-14           [-1, 64,

In [82]:
model_name = "CLIP:RN50"
model = get_model(model_name, type="wmap", roi_pooling=True)

In [85]:
# Load image and extract landmarks
image = "../misc/john.jpg"
rois = extract_rois(image)
boxes = [box for box, _ in rois]
# Convert to tensor
boxes_tensor = torch.tensor(boxes, dtype=torch.float32)  # shape (N_rois, 4)
# Add batch indices (0 if only 1 image)
batch_indices = torch.zeros((boxes_tensor.shape[0], 1), dtype=torch.float32)
# Concatenate batch index
boxes_tensor = torch.cat([batch_indices, boxes_tensor], dim=1)  # (N_rois, 5)
print(boxes_tensor.shape)  # should be (N_rois, 5)

Found 1 face(s) in this photograph.
torch.Size([9, 5])


In [86]:
from PIL import Image
import numpy as np

# Load image
image = Image.open('../misc/john.jpg')
# Resize to 224x224
image = np.array(image.resize((224, 224)))
image_tensor = torch.from_numpy(image).permute(2, 0, 1).float()  # C,H,W
image_tensor /= 255.0
image_tensor = image_tensor[np.newaxis, :]
image_tensor.shape

torch.Size([1, 3, 224, 224])

In [87]:
image_input = torch.randn(1, 3, 224, 224)
loss_map = torch.randn(1, 4, 32, 32)

with torch.no_grad():
    logits = model(image_tensor, loss_map, rois=boxes_tensor)
    print(logits.shape)

torch.Size([1, 2])
